# Pre-processing

In [1]:
import ibis
from ibis import _

conn = ibis.duckdb.connect("tmp", extensions=["spatial"])
ca_parquet = "https://data.source.coop/cboettig/ca30x30/ca_areas.parquet"
# or use local copy:
ca_parquet = "/home/rstudio/source.coop/cboettig/ca30x30/ca_areas.parquet"


In [2]:
buffer = -0.00001

tbl = (
    conn.read_parquet(ca_parquet)
    .cast({"SHAPE": "geometry"})
    .rename(geom = "SHAPE")
   # .filter(_.UNIT_NAME == "Angeles National Forest")
    .filter(_.reGAP < 3) 
)
tbl_2023 = tbl.filter(_.Release_Year == 2023).mutate(geom=_.geom.buffer(buffer))
tbl_2024 = tbl.filter(_.Release_Year == 2024)
intersects = tbl_2024.anti_join(tbl_2023, _.geom.intersects(tbl_2023.geom))

new2024 = intersects.select("OBJECTID").mutate(established = 2024)

In [3]:
ca = (conn
      .read_parquet(ca_parquet)
      .cast({"SHAPE": "geometry"})
      .mutate(area = _.SHAPE.area())
      .filter(_.Release_Year == 2024)
      .filter(_.reGAP < 3)
      .left_join(new2024, "OBJECTID")
      .mutate(established=_.established.fill_null(2023))
      .mutate(geom = _.SHAPE.convert("epsg:3310","epsg:4326"))
      .rename(name = "UNIT_NAME", access_type = "ACCESS_TYP", manager = "MNG_AGNCY",
              manager_type = "MNG_AG_LEV", id = "OBJECTID", type = "TYPE")
      .select(_.established, _.reGAP, _.name, _.access_type, _.manager, _.manager_type,
              _.Easement, _.Acres, _.id, _.type, _.geom)
     )
ca.execute().to_parquet("ca2024.parquet")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# Testing & visualization

`ca2024.parquet()` now contains all we need.  The code below illustrates some quick examples of the kinds of visualizations and summaries we might want to compute with this data.  


In [1]:
import leafmap.maplibregl as leafmap
import ibis
from ibis import _
conn = ibis.duckdb.connect(extensions=["spatial"])

ca2024 = conn.read_parquet("ca2024.parquet")

Using difference (overlap) instead:

In [6]:
# compute some summary tables:

(ca2024
 .filter(_.established == 2024)
 .filter(_.manager_type == "State")
 .group_by(_.manager, _.manager_type)
 .agg(area = _.Acres.sum())
 .order_by(_.area.desc())
 .execute()
)

,manager,manager_type,area
0,California Department of Fish and Wildlife,State,42086.259379
1,California Department of Parks and Recreation,State,17931.321473
2,California Tahoe Conservancy,State,4803.250929
3,California Department of Water Resources,State,2248.610289
4,University of California,State,1860.854444
5,California Department of Forestry and Fire Pro...,State,1089.029581
6,Coachella Valley Mountains Conservancy,State,119.907070
7,California State Lands Commission,State,109.016475
8,California State Coastal Conservancy,State,97.314705
9,California State University Sonoma,State,38.760956


In [7]:
gdf = ca2024.filter(_.manager == "California Department of Parks and Recreation", _.established== 2024).execute()

established = {'property': 'established',
               'type': 'categorical',
               'stops': [
                   [2023, "#26542C80"], 
                   [2024, "#F3AB3D80"]]
              }
paint = {"fill-color": established}

m = leafmap.Map(style="positron")
m.add_gdf(gdf,layer_type="fill", name = "CA 30x30", paint = paint)
m.add_layer_control()
m

Map(height='600px', map_options={'bearing': 0, 'center': (0, 20), 'pitch': 0, 'style': 'https://basemaps.carto…